In [1]:
%pylab nbagg

Populating the interactive namespace from numpy and matplotlib


In [2]:
from pathlib import Path
from os import listdir
import pandas as pd
from datetime import datetime, timedelta
from importlib import reload
import utils
import pickle
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from scipy.stats import mode

In [3]:
from fastai.tabular import *

In [4]:
reload(utils)

<module 'utils' from 'C:\\Users\\efreiling\\Desktop\\new_york_311\\Notebooks\\utils.py'>

# Load the data
- We are only dealing with 10% of the data for memory and time purposes

In [5]:
data_folder = Path('../../data/new_york_311')
data_path = data_folder / 'data_311.csv'

In [6]:
orig_data = pd.read_csv(data_path)

C:\Users\efreiling\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Data Processing

### Drop Certain Columns

One of the main columns that would be useful is resolution description. Depending on the scenario or application this could be considered to have leakage. However, if we were to include it in the features, we can try several approaches:
- Drop stop words
- Convert words to vectors using word2vec, take the average of the vectors
- Sklearn HashingVectorizer
- Use Tf-Idf to find important words, use word2vec on top N words

In [34]:
drop_cols = [
    'unique_key',                     # If interested in finding hidden leakage, investigate this column
    'agency_name',                    # Redundant to agancy
    'descriptor',                     # This gives away the complaint type, leakage
    'incident_address',               # Didnt want street numbers
    'bbl',                            # We already have enough goelocaions
    'location',                       # Redundant to Lat and Lng
    'resolution_action_updated_date', # caused errors
    'resolution_description'          # Probably very useful but not enough time to use and maybe leakage
]


In [35]:
df = utils.drop_useless_cols(orig_data, drop_cols)       

### Filter Rows
- Only use rows that contain the 130 complaint types
- Filter rows that are all NaN

In [36]:
df = utils.filter_rows(df)

### Define Continuous, Categorical, Date Columns

# Data Product 

Instead of building a model on the complaint type, I think it would be interesting to build a model on predicting the time elapsed between date created and closed date. I think this would be more product focused. Given a complaint, a user/manager can use the model to determine if it is going to take a long time to close. Maybe resources could be better allocated for quicler resolutions.

### Complaint type is a Cat Col

In [37]:
cont_cols = [
    'x_coordinate_state_plane',
    'y_coordinate_state_plane',
    'latitude',
    'longitude'
]
cat_cols = [
    'complaint_type',
    'agency',
    'borough',
    'location_type',
    'incident_zip',
    'street_name',
    'cross_street_1',
    'cross_street_2',
    'intersection_street_1',
    'intersection_street_2',
    'address_type',
    'city',
    'landmark',
    'facility_type',
    'status',
    'community_board',
    'open_data_channel_type',
    'park_facility_name',
    'park_borough',
    'vehicle_type',
    'taxi_company_borough',
    'taxi_pick_up_location',
    'bridge_highway_name',
    'bridge_highway_direction',
    'road_ramp',
    'bridge_highway_segment',
]
date_cols = [
    'created_date',
    'closed_date',
    'due_date',
]


### Process All Date Columns Except Closed Date
- Add columns for time elapsed between dates
- Add features for day of week, end of year, etc

In [38]:
# Convert all date cols to datetimes
for dc in date_cols:
    df[dc] = pd.to_datetime(df[dc])

# add columns for time elapsed between dates
df['time_to_close'] = (df['closed_date']-df['created_date']).astype('timedelta64[h]')
df['due_len'] = (df['due_date']-df['created_date']).astype('timedelta64[h]')
# df['time_over'] = (df['due_date']-df['closed_date']).astype('timedelta64[h]')
cont_cols += ['time_to_close', 'due_len']
#df, cont_cols = utils.create_date_lengths(df, cont_cols)
date_cols.remove('closed_date')
# Add date boolean features, day of week, end of year, etc
for d in date_cols:
    add_datepart(df, d, drop=True)

# Keep track of created categorical columns
cat_cols += list(set(df.columns) - set(cont_cols) - set(cat_cols) - set(dep_var))

In [39]:
cat_cols.remove('closed_date')
df = df.drop(columns=['closed_date'])
dep_var = ['time_to_close']

### Process Continuous Columns
- Fill in missing values with the median of the column, then add another feature that idicates which rows had missing values
- Normalize Z-score

In [41]:
df, cont_cols, cat_cols = utils.process_cont_cols(df, cont_cols, cat_cols)

In [44]:
df = df.drop(columns=['time_to_close_missing'])
cat_cols.remove('time_to_close_missing')

### Process Categorical Columns
- Ordinal Encoding for columns with number of classes greater than 20
- One hot encoding for columns with number of classes 20 or less

In [45]:
df, cat_cols, label_encoders = utils.process_cat_cols(df, cat_cols)

### Split Dataframe into Features and Target

In [47]:
y = df[dep_var[0]]
x = df.drop(columns=dep_var)

# Train a Random Forest Regressor and Compare to Naive Solution

In [48]:
seed = 45
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=seed)
clf = RandomForestRegressor(max_depth=12, random_state=seed)

In [49]:
clf.fit(train_x, train_y)

C:\Users\efreiling\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=12,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=45, verbose=0,
                      warm_start=False)

In [50]:
preds = clf.predict(test_x)
rf_error = np.mean(np.abs(preds - np.array(test_y)))
naive_error = np.mean(np.abs(np.array(test_y)))
print('Accuracy')
print(f'Random Forest: {rf_error}, Naive: {naive_error}')

Accuracy
Random Forest: 0.02967872684001366, Naive: 0.03878168471282954


In [52]:
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
list(zip(train_x.columns[indices], importances[indices]))

[('due_len', 0.24175687678100155),
 ('due_Elapsed', 0.21411207107336191),
 ('created_Elapsed', 0.100418148318213),
 ('complaint_type', 0.085779436102114),
 ('agency=DOHMH', 0.07981887371700853),
 ('status=Assigned', 0.03236341752537319),
 ('created_Week', 0.01566317404512454),
 ('created_Dayofyear', 0.015444841596251091),
 ('street_name', 0.014778517326053016),
 ('location_type', 0.01444594300958986),
 ('due_Year=2012.0', 0.0115664341941727),
 ('due_Day', 0.0112185801157408),
 ('cross_street_2', 0.011030338119510286),
 ('created_Day', 0.01082944547901107),
 ('created_Year=2012', 0.008499906754067648),
 ('community_board', 0.008314506155426364),
 ('longitude', 0.007647092675971841),
 ('cross_street_1', 0.007301724373272027),
 ('x_coordinate_state_plane', 0.00729601556435464),
 ('incident_zip', 0.007209613546275584),
 ('due_Week', 0.006579450375785611),
 ('due_Dayofyear', 0.006321154396561096),
 ('created_Month=3', 0.00576884005372389),
 ('latitude', 0.004973083667598879),
 ('y_coordinat